Import required libraries

In [86]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import glob
import os

Load our datasets

In [87]:
# Get our file path for the .csv files
file_paths = [f for f in os.listdir("./Dataset/") if f.endswith('.csv')]

# Define list for our data frames
li = []

# Loop through our file paths and read the .csv files into a data frame
for filename in file_paths:
    filename = "./Dataset/" + filename
    
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# Concatenate our data frames into one central data frame
match_df = pd.concat(li, axis=0, ignore_index=True)

# Have a look at our data frame
match_df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/2017,Arsenal,Leicester,4,3,H,2,2,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,12/08/2017,Brighton,Man City,0,2,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,12/08/2017,Chelsea,Burnley,2,3,A,0,3,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,12/08/2017,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,12/08/2017,Everton,Stoke,1,0,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
# Get our team overall ratings by season
team_overall_ratings_df = pd.read_csv("./Dataset/seasonal_ratings/seasonal_team_rating.csv", index_col=None, header=0)

# Have a look at our data frame
team_overall_ratings_df.head()

,Season,Team,Rating
0,2022-2023,Man City,85
1,2022-2023,Liverpool,84
2,2022-2023,Arsenal,82
3,2022-2023,Chelsea,82
4,2022-2023,Man United,82


Re-format dataset structure / namings etc. and join both datasets together

In [89]:
# Drop the "Div" column as this is just indicates what football division the data is for
# not relevant for our analysis as we already know it is looking at the Premier League
match_df = match_df.drop(["Div"], axis=1)

In [90]:
# Add season column to see what season the match was played in

# Convert Date column from object to datetime
match_df["Date"] = pd.to_datetime(match_df["Date"], format='%d/%m/%Y')

# Define the date ranges for each season from 2017/18 to 2022/23
seasons = {
    '2017-2018': ('2017-08-11', '2018-05-13'),
    '2018-2019': ('2018-08-10', '2019-05-12'),
    '2019-2020': ('2019-08-09', '2020-07-26'),
    '2020-2021': ('2020-09-12', '2021-05-23'),
    '2021-2022': ('2021-08-13', '2022-05-22'),
    '2022-2023': ('2022-08-05', '2023-05-28')
}

# Define a function to get the season given a date
def get_season(date):
    
    for season, (start_date, end_date) in seasons.items():
        if pd.to_datetime(start_date) <= date <= pd.to_datetime(end_date):
            return season
        
    return 'Unknown'  # or whatever you want to return when the date doesn't fall into any of the defined seasons

match_df["Season"] = match_df["Date"].apply(get_season)

In [91]:
# Check the seasons have been added correctly by checking each season has the same number of matches (380)
match_df["Season"].value_counts()

Season
2017-2018    380
2018-2019    380
2019-2020    380
2020-2021    380
2021-2022    380
2022-2023    380
Name: count, dtype: int64

In [92]:
# Add the away and home team overall ratings to the match data frame

# Merge to get the home team ratings
match_df = pd.merge(match_df, team_overall_ratings_df, how='left', left_on=['Season', 'HomeTeam'], right_on=['Season', 'Team'])
match_df = match_df.rename(columns={'Rating': 'home_rating_overall'})
match_df = match_df.drop('Team', axis=1)  # drop the extra 'team' column

# Merge to get the away team ratings
match_df = pd.merge(match_df, team_overall_ratings_df, how='left', left_on=['Season', 'AwayTeam'], right_on=['Season', 'Team'])
match_df = match_df.rename(columns={'Rating': 'away_rating_overall'})
match_df = match_df.drop('Team', axis=1)  # drop the extra 'team' column

match_df

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,Season,home_rating_overall,away_rating_overall
0,2017-08-11,Arsenal,Leicester,4,3,H,2,2,D,M Dean,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,82,78
1,2017-08-12,Brighton,Man City,0,2,A,0,0,D,M Oliver,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,75,84
2,2017-08-12,Chelsea,Burnley,2,3,A,0,3,A,C Pawson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,83,77
3,2017-08-12,Crystal Palace,Huddersfield,0,3,A,0,2,A,J Moss,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,76,74
4,2017-08-12,Everton,Stoke,1,0,H,1,0,H,N Swarbrick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,79,76
5,2017-08-12,Southampton,Swansea,0,0,D,0,0,D,M Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,76,75
6,2017-08-12,Watford,Liverpool,3,3,D,2,1,H,A Taylor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,77,81
7,2017-08-12,West Brom,Bournemouth,1,0,H,1,0,H,R Madley,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,76,76
8,2017-08-13,Man United,West Ham,4,0,H,1,0,H,M Atkinson,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,83,77
9,2017-08-13,Newcastle,Tottenham,0,2,A,0,0,D,A Marriner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-2018,75,83
